<a href="https://colab.research.google.com/github/YordanIg/learning-ML/blob/main/classification_cnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Perform image classification with a CNN.

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
if torch.cuda.is_available():
  print("CUDA available, using GPU.")
  torch.device("cuda")
else:
  print("CUDA unavailable, using CPU.")
  torch.device("cpu")

CUDA available, using GPU.


In [ ]:
# Load training and testing datasets.
to_tensor = transforms.ToTensor()
trainset  = torchvision.datasets.MNIST(root='./data', train=True,
                                       download=True, transform=to_tensor)
testset   = torchvision.datasets.MNIST(root='./data', train=False,
                                       download=True, transform=to_tensor)

100%|██████████| 9.91M/9.91M [00:01<00:00, 5.17MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 134kB/s]
100%|██████████| 1.65M/1.65M [00:01<00:00, 1.28MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 7.76MB/s]


In [ ]:
# Instantiate dataloaders.
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)
testloader  = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False)

In [ ]:
# Set up the CNN.
class CNN(nn.Module):
  def __init__(self):
    super().__init__()
    # Convolutional layer 1: (1,28,28) -> (16,28,28) -> (16,14,14).
    self.conv1 = nn.Conv2d(in_channels=1, out_channels=16,
                           kernel_size=3, padding=1)
    self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)

    # Convolutional layer 2: (16,14,14) -> (32,14,14) -> (32,7,7).
    self.conv2 = nn.Conv2d(in_channels=16, out_channels=32,
                           kernel_size=3, padding=1)
    self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)

    # FFN.
    self.fc1 = nn.Linear(in_features=32*7*7, out_features=128)
    self.fc2 = nn.Linear(in_features=128, out_features=10)

  def forward(self, x):
    # Convolutional layer 1.
    x = F.relu(self.conv1(x))
    x = self.pool1(x)

    # Convolutional layer 2.
    x = F.relu(self.conv2(x))
    x = self.pool2(x)

    # FFN.
    x = x.view(-1,32*7*7)
    x = F.relu(self.fc1(x))
    x = self.fc2(x)         # Do not apply ReLU as will use CELoss.
    return x


In [ ]:
# Test to see if network works on a batch of 3 noise images of the correct
# shape and with 1 colour channel each.
x = torch.rand((3,1,28,28))
model = CNN()
model(x)

tensor([[-0.0420, -0.0165, -0.0618, -0.0421, -0.0830, -0.0439,  0.0086,  0.0790,
         -0.0780, -0.0431],
        [-0.0358, -0.0143, -0.0528, -0.0487, -0.0873, -0.0346,  0.0164,  0.0711,
         -0.0790, -0.0403],
        [-0.0303, -0.0106, -0.0463, -0.0425, -0.0803, -0.0375,  0.0206,  0.0846,
         -0.0798, -0.0446]], grad_fn=<AddmmBackward0>)

In [ ]:
# Set up loss.
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [ ]:
# Training loop.
num_epochs = 8
for epoch in range(num_epochs):
  running_loss = 0
  for i, data in enumerate(trainloader):
    # Zero the gradients.
    optimizer.zero_grad()
    # Compute the loss.
    x, y = data
    prediction = model(x)
    loss = criterion(prediction, y)
    # Compute the gradients and take a step.
    loss.backward()
    optimizer.step()
    # Update the running loss.
    running_loss += loss.item()
    if i%200==0 and i!=0:
      print(f"Epoch {epoch+1}, Batch {i+1}: avg loss since last print {running_loss/200:.3f}")
      running_loss = 0
print("Training complete")

Epoch 1, Batch 201: avg loss since last print 0.569
Epoch 1, Batch 401: avg loss since last print 0.168
Epoch 1, Batch 601: avg loss since last print 0.119
Epoch 1, Batch 801: avg loss since last print 0.093
Epoch 2, Batch 201: avg loss since last print 0.067
Epoch 2, Batch 401: avg loss since last print 0.068
Epoch 2, Batch 601: avg loss since last print 0.063
Epoch 2, Batch 801: avg loss since last print 0.062
Epoch 3, Batch 201: avg loss since last print 0.045
Epoch 3, Batch 401: avg loss since last print 0.045
Epoch 3, Batch 601: avg loss since last print 0.046
Epoch 3, Batch 801: avg loss since last print 0.042
Epoch 4, Batch 201: avg loss since last print 0.033
Epoch 4, Batch 401: avg loss since last print 0.032
Epoch 4, Batch 601: avg loss since last print 0.031
Epoch 4, Batch 801: avg loss since last print 0.034
Epoch 5, Batch 201: avg loss since last print 0.022
Epoch 5, Batch 401: avg loss since last print 0.031
Epoch 5, Batch 601: avg loss since last print 0.027
Epoch 5, Bat

In [ ]:
total = 0
correct = 0
with torch.no_grad():
  for data in testloader:
    x, y = data
    outputs = model(x)
    _, predictions = torch.max(outputs.data, 1)
    total += y.size(0)
    correct += (predictions == y).sum().item()  # See example for explanation.

accuracy = 100 * correct/total
print(f"Model accuracy is {accuracy}")

Model accuracy is 98.99


In [ ]:
## Tweaking hyperparameters.
# When run with 512 hidden FC nodes, get
# >> Epoch 8, Batch 801: avg loss since last print 0.012
# >> Model accuracy is 98.99
# When run with 256 hidden FC nodes, get
# >> Epoch 8, Batch 801: avg loss since last print 0.013
# >> Model accuracy is 98.99
# When run with 128 hidden FC nodes, get
# >> Model accuracy is 98.9
# When run with 64 hidden FC nodes, get
# >> Model accuracy is 99.01
# When run with 32 hidden FC nodes, get
# >> Model accuracy is 98.8

# Beginning to get lower performance. Doesn't look like the 512 version is
# overfitting.

In [ ]:
# Example of what the (predictions == y).sum().item() line does.
# This simply compares the two vectors element-wise, sums the number of "True"s
# and converts it into an integer.
a = torch.tensor([1, 2, 3, 4])
b = torch.tensor([5, 2, 6, 4])
print(a == b)
print((a == b).sum())
print((a == b).sum().item())

tensor([False,  True, False,  True])
tensor(2)
2
